用窗函数法设计一个 $h(n)$ 偶对称的线性相位FIR带通滤波器，给定下阻带截止频率 $f_{st_{1}}=2kHz$，上阻带截止频率 $f_{st_{2}}=6kHz$，通带下截止频率为 $f_{p_{1}}=3kHz$，通带上截止频率为 $f_{p_{2}}=5kHz$，阻带最小衰减为 $A_{s}=55dB$，抽样频率为$f_{s}=20kHz$。<br/>
理想带通滤波器的频率响应和单位抽样响应分别为<br/>
$H_{d}(e^{j\omega})=\begin{cases}  & \text{} e^{-j\omega\tau},\omega_{1}\le\left | \omega  \right | \le \omega _{2}   \\  & \text{} 0,0\le \left | \omega  \right | < \omega _{1} ,\omega _{2} < \left | \omega  \right |\le \pi  \end{cases}$ <br/>
$h_{d}(n)=\frac{1}{2\pi }\int_{-\omega _{2} }^{-\omega _{1} }e^{-j\omega \tau }e^{j\omega n} d\omega +\frac{1}{2\pi }\int_{\omega _{1} }^{\omega _{2} }e^{-j\omega \tau }e^{j\omega n} d\omega= \begin{cases}  & \text{} \frac{\sin [\omega _{2}(n-\tau )]-\sin [\omega _{1}(n-\tau ) ]}{\pi (n-\tau )} ,n\ne \tau   \\  & \text{} (\omega _{2}-\omega _{1})/\pi , n=\tau (\tau 为整数时)\end{cases}$<br/>
为满足线性相位，$\tau =(N-1)/2$；程序如下


In [ ]:
#导入使用的库
import numpy as np;from math import *
from scipy import signal,fft
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

#滤波器参数
fs = 20000; #抽样频率
wp1 = 2*pi*3000/fs;wp2 = 2*pi*5000/fs #通带截止频率
ws1 = 2*pi*2000/fs;ws2 = 2*pi*6000/fs #阻带截止频率
w1 = (wp1+ws1)/2;w2 = (wp2+ws2)/2 
Bt = np.array([np.abs(wp1-ws1),np.abs(wp2-ws2)]);Bt = Bt.min() #过渡带宽

#由阻带衰减As = 55，可以确定窗形状为布拉克曼窗
N = np.ceil((11*pi)/Bt)+1;N = int(N+(N+1)%2)#滤波器长度点数（取奇数）
wn = signal.windows.blackman(N) #汉宁窗的wn值

#理想低通滤波器的单位采样响应
t = int((N-1)/2)
n1 = np.arange(N);n1 = np.delete(n1,t)
hd = (np.sin(w2*(n1-t))-np.sin(w1*(n1-t)))/(pi*(n1-t))
hd = np.insert(hd,t,(w2-w1)/pi)

#线性相位FIR滤波器
h = hd*wn;N0 = N*1000
He = np.abs(fft.fft(h,N0));He = He/np.max(He)
Ar = 20*np.log10(He);N1 = int(N0/2);f = np.linspace(0,1,N1)

#绘制滤波器的幅度响应
fig,ax = plt.subplots();ax.plot(f,Ar[:N1]);ax.grid()
ax.set_title('使用布拉克曼窗设计的FIR滤波器的幅度响应');ax.set_xlabel('k')
ax.set_xlabel(r'$ \omega / \pi $')
ax.set_ylabel(r'$ 20log_{10}| H (e^{j \omega}) | $')
ax.set_xlim([0,1]);ax.set_ylim([-100,1])
ax.xaxis.set_major_locator(MaxNLocator(11))
ax.yaxis.set_major_locator(MaxNLocator(11))
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来显示负号
fig.savefig('./fir_window2.png',dpi=500)

根据上图可以看出，各项指标均得到满足，FIR滤波器设计完成。